# Experiment 4.5 – Real Data

## Setup

In [1]:
using Glob
using CSV
using DataFrames
using Random

In [2]:
using Gadfly
import Cairo, Fontconfig

In [3]:
ENV["COLUMNS"] = 1000;

## Style

In [4]:
function get_match_ratio_colorid(x)
    if x == 1.0
        return 4
    elseif x >= 0.85
        return 3
    elseif x >= 0.5
        return 2
    else
        return 1
    end
end;

In [5]:
colors = [colorant"red", colorant"darkblue", colorant"deepskyblue", colorant"green"]

In [6]:
defaultfont = "CMU Serif";
fontstyles = (
    key_title_font = defaultfont,
    key_label_font = defaultfont,
    minor_label_font = defaultfont,
    major_label_font = defaultfont,
    key_title_font_size = 14px,
    key_label_font_size = 12px,
    minor_label_font_size = 14px,
    major_label_font_size = 16px,
);

## Load Data

In [7]:
fns = glob("*.csv", "../results/experiment_4_5a/");
results_list = [DataFrame(CSV.File(fn)) for fn in fns];
results = vcat(results_list...);

In [8]:
sort!(results, :match_ratio, rev=true);

In [9]:
mean(x) = sum(x) / length(x);

In [10]:
results_mean = combine(groupby(results, [:N, :ρe]), [
    :match_ratio => mean => :match_ratio,
    :alignment_strength_1 => mean => :alignment_strength_1,
]);

In [11]:
results.match_color = map(get_match_ratio_colorid, results.match_ratio);
results_mean.match_color = map(get_match_ratio_colorid, results_mean.match_ratio);

## Figures

In [12]:
figurespath = "./figures_45a/tex/"
if !isdir(figurespath) mkpath(figurespath) end;

In [13]:
function plot_fig1(df, v="v1"; display=true, save=false)
    @assert length(unique(df.N)) == 1
    
    if v == "v1"
        pointstyles = (
            point_size=2px,
            highlight_width=0px,
        )
        figtype = "discretecolor_all"
    else
        pointstyles = (
            highlight_width=0px,
        )
        figtype = "discretecolor_mean"
    end
    if !isdir(joinpath(figurespath, figtype)) mkpath(joinpath(figurespath, figtype)) end
    
    plt = plot(
        df,
        x=:ρe, y=:alignment_strength_1, color=:match_color,
        Geom.point,

        Guide.xlabel("Parameter (ρ)"),
        Guide.ylabel("Alignment Strength"),
        Guide.title("Real Data (n=$(df.N[1]))"),

        Scale.color_discrete_manual(colors..., levels=[1,2,3,4]),
        Guide.colorkey(title="Match Ratio", labels=["&lt;50%", "≥50%", "≥85%", "100%"]),

        Guide.xticks(ticks=0:0.25:1),
        Guide.yticks(ticks=0:0.25:1),

        style(
            ;fontstyles...,
            pointstyles...,
        ),
    )
    
    if display
        plt |> SVG(16cm, 10cm)
    end
    if save
        n_str = lpad(df.N[1]+1, 5, "0")
        plt |> PGF(joinpath(figurespath, figtype, "exp45a_fig1_$(v)_$(n_str).tex"), 16cm, 10cm, texfonts=true)
    end
    
    return plt
end;

In [14]:
function plot_fig2(df, v="v1"; display=true, save=false)
    @assert length(unique(df.N)) == 1
    
    if v == "v1"
        pointstyles = (
            point_size=2px,
            highlight_width=0px,
        )
        figtype = "continuouscolor_all"
    else
        pointstyles = (
            highlight_width=0px,
        )
        figtype = "continuouscolor_mean"
    end
    if !isdir(joinpath(figurespath, figtype)) mkpath(joinpath(figurespath, figtype)) end
    
    plt = plot(
        df,
        x=:ρe, y=:alignment_strength_1, color=:match_ratio,
        Geom.point,

        Guide.xlabel("Parameter (ρ)"),
        Guide.ylabel("Alignment Strength"),
        Guide.title("Real Data (n=$(df.N[1]))"),

        Guide.colorkey(title="Match Ratio"),

        Guide.xticks(ticks=0:0.25:1),
        Guide.yticks(ticks=0:0.25:1),

        style(
            ;fontstyles...,
            pointstyles...,
        ),
    )
    
    if display
        plt |> SVG(16cm, 10cm)
    end
    if save
        n_str = lpad(df.N[1]+1, 5, "0")
        plt |> PGF(joinpath(figurespath, figtype, "exp45a_fig2_$(v)_$(n_str).tex"), 16cm, 10cm, texfonts=true)
    end
    
    return plt
end;

In [15]:
for N in sort(unique(results.N))
    @show N
    r = filter(row -> row.N == N, results)
    r = r[randperm(nrow(r)),:]
    r_mean = filter(row -> row.N == N, results_mean)
    
    plot_fig1(r, "v1", display=false, save=true)
    plot_fig1(r_mean, "v2", display=false, save=true)
    plot_fig2(r, "v1", display=false, save=true)
    plot_fig2(r_mean, "v2", display=false, save=true)
end

N = 70
N = 94
N = 107
N = 139
N = 194
N = 277
N = 349
N = 445
N = 582
N = 832
N = 1215
N = 1874
N = 3230
